# Event Management with Neon CRM SDK

This notebook demonstrates comprehensive event management operations using the Neon CRM SDK.

⚠️ **SAFETY NOTE**: This notebook contains only read-only operations for safety.
Database-modifying operations are commented out to prevent accidental changes.

In [1]:
import os
from datetime import datetime, timedelta
from neon_crm import NeonClient
import pandas as pd

# Initialize the client
client = NeonClient(
    org_id=os.getenv('NEON_ORG_ID'),
    api_key=os.getenv('NEON_API_KEY'),
)

print(f"Connected to Neon CRM - Environment: {client.environment}")

Connected to Neon CRM - Environment: production


## 1. List Events (Read-Only)

Retrieve and display events from your organization.

In [2]:
# List events (safe read-only operation)
print("📅 Retrieving events...")

events = list(client.events.list(page_size=200, limit=10))

print(f"✓ Retrieved {len(events)} events")

if events:
    # Display event information
    for i, event in enumerate(events[:5], 1):
        print(f"\n{i}. {event.get('name', 'Unnamed Event')}")
        print(f"   ID: {event.get('id')}")
        print(f"   Start Date: {event.get('startDate', 'N/A')}")
        print(f"   Status: {event.get('eventStatus', 'N/A')}")
        print(f"   Category: {event.get('category', {}).get('name', 'N/A')}")
else:
    print("No events found in your organization.")

📅 Retrieving events...
✓ Retrieved 10 events

1. ECC Democrats Annual Holiday Party
   ID: 1
   Start Date: 2019-12-19
   Status: N/A
   Category: N/A

2. SPRING CELEBRATION USE
   ID: 761
   Start Date: 2023-04-14
   Status: N/A
   Category: N/A

3. Summer Picnic 2023
   ID: 763
   Start Date: 2023-08-17
   Status: N/A
   Category: N/A

4. ECCD Annual Holiday Party
   ID: 1266
   Start Date: 2023-12-14
   Status: N/A
   Category: N/A

5. 2024 Spring Celebration
   ID: 1268
   Start Date: 2024-04-15
   Status: N/A
   Category: N/A


✓ Retrieved 10 events

1. ECC Democrats Annual Holiday Party
   ID: 1
   Start Date: 2019-12-19
   Status: N/A
   Category: N/A

2. SPRING CELEBRATION USE
   ID: 761
   Start Date: 2023-04-14
   Status: N/A
   Category: N/A

3. Summer Picnic 2023
   ID: 763
   Start Date: 2023-08-17
   Status: N/A
   Category: N/A

4. ECCD Annual Holiday Party
   ID: 1266
   Start Date: 2023-12-14
   Status: N/A
   Category: N/A

5. 2024 Spring Celebration
   ID: 1268
   Start Date: 2024-04-15
   Status: N/A
   Category: N/A


## 2. Event Search Operations (Read-Only)

Search for events using specific criteria.

In [3]:
# Search for upcoming events
print("🔍 Searching for upcoming events...")

search_request = {
    "searchFields": [
        {"field": "Event Archived", "operator": "EQUAL", "value": "false"}
    ],
    "outputFields": [
        "Event ID", "Event Name", "Event Start Date", "Event End Date", 
        "Event Category", "Event Registration Status"
    ],
    "pagination": {"currentPage": 0, "pageSize": 200}
}

try:
    search_results = list(client.events.search(search_request, validate=False))
    
    print(f"✓ Found {len(search_results)} non-archived events")
    
    if search_results:
        # Convert to DataFrame for better display
        df = pd.DataFrame(search_results)
        print("\n📊 Events Summary:")
        print(df.head())
        
        # Display some key statistics
        print(f"\nTotal Events Found: {len(search_results)}")
        
        # Show unique event categories if available
        if 'Event Category' in df.columns:
            unique_categories = df['Event Category'].value_counts()
            print(f"\nEvent Categories Distribution:")
            for category, count in unique_categories.head().items():
                print(f"  {category}: {count}")
                
    else:
        print("No non-archived events found.")
        
except Exception as e:
    print(f"❌ Search failed: {e}")
    print("Note: This may be due to field name validation issues.")
    
    # Try a simpler search as fallback
    print("\n🔄 Trying simpler search...")
    try:
        simple_search = {
            "searchFields": [
                {"field": "Event Name", "operator": "NOT_BLANK", "value": ""}
            ],
            "outputFields": [
                "Event ID", "Event Name", "Event Start Date"
            ],
            "pagination": {"currentPage": 0, "pageSize": 50}
        }
        
        simple_results = list(client.events.search(simple_search, validate=False))
        print(f"✓ Simple search found {len(simple_results)} events with names")
        
        if simple_results:
            simple_df = pd.DataFrame(simple_results)
            print("\n📊 Simple Search Results:")
            print(simple_df.head())
        
    except Exception as simple_e:
        print(f"❌ Simple search also failed: {simple_e}")

🔍 Searching for upcoming events...


2025-10-04 15:28:15 - neon_crm.resource.customFields - INFO - Custom field 'Event Archived' not found in category 'Event'. Did you mean: Menu Choices, Meal Choice? Or perhaps you were looking for: Indicate what category and food you plan to share., Indicate what category and food you plan to bring.?
2025-10-04 15:28:15 - neon_crm.resource.customFields - INFO - Custom field 'Event Category' not found in category 'Event'. Maybe you were looking for: Indicate what category and food you plan to share., Indicate what category and food you plan to bring., Menu Choices?
2025-10-04 15:28:16 - neon_crm.resource.customFields - INFO - Custom field 'Event Registration Status' not found in category 'Event'. Maybe you were looking for: Indicate what category and food you plan to share., Indicate what category and food you plan to bring., Menu Choices?


❌ Search failed: HTTP 400: [Code 25] Search value is invalid.; [Code 28] Search output field name is invalid; [Code 28] Search output field name is invalid
Note: This may be due to field name validation issues.

🔄 Trying simpler search...
✓ Simple search found 12 events with names

📊 Simple Search Results:
  Event Start Date                      Event Name Event ID
0       2024-12-12  2024 ECCD Annual Holiday Party     1279
1       2024-04-15         2024 Spring Celebration     1268
2       2025-05-03         2025 Spring Celebration     1283
3       2025-08-21            ANNUAL SUMMER PICNIC     1274
4       2025-08-21       ANNUAL SUMMER PICNIC 2025     1289


2025-10-03 13:36:07 - neon_crm.resource.customFields - INFO - Custom field 'Event Registration Status' not found in category 'Event'. Maybe you were looking for: Indicate what category and food you plan to share., Indicate what category and food you plan to bring., Menu Choices?


❌ Search failed: HTTP 400: [Code 25] Search value is invalid.; [Code 28] Search output field name is invalid; [Code 28] Search output field name is invalid
Note: This may be due to field name validation issues.

🔄 Trying simpler search...


✓ Simple search found 12 events with names

📊 Simple Search Results:
  Event Start Date                      Event Name Event ID
0       2024-12-12  2024 ECCD Annual Holiday Party     1279
1       2024-04-15         2024 Spring Celebration     1268
2       2025-05-03         2025 Spring Celebration     1283
3       2025-08-21            ANNUAL SUMMER PICNIC     1274
4       2025-08-21       ANNUAL SUMMER PICNIC 2025     1289


## 3. Event Categories (Read-Only)

Retrieve available event categories.

In [4]:
# Get event categories
print("📂 Retrieving event categories...")

try:
    categories = client.events.get_categories()
    
    print(f"✓ Found {len(categories)} event categories")
    
    for category in categories:
        print(f"  • {category.get('name', 'Unnamed Category')} (ID: {category.get('id')})")
        
except Exception as e:
    print(f"❌ Failed to retrieve categories: {e}")

📂 Retrieving event categories...
✓ Found 1 event categories
  • Fundraising (ID: 1)


## 4. Event Field Information (Read-Only)

Get available search and output fields for events.

In [5]:
# Get event search fields
print("🔍 Retrieving event search fields...")

try:
    search_fields = client.events.get_search_fields()
    
    print(f"✓ Found {len(search_fields)} search fields")
    print("\nSample search fields:")
    for field in search_fields['standardFields']:
        print(f"  • {field}")
        
    for field in search_fields['customFields']:
        print(f"  • {field}")
except Exception as e:
    print(f"❌ Failed to retrieve search fields: {e}")

🔍 Retrieving event search fields...
✓ Found 2 search fields

Sample search fields:
  • {'fieldName': 'Event', 'operators': ['BLANK', 'NOT_BLANK', 'EQUAL', 'NOT_EQUAL', 'IN_RANGE', 'NOT_IN_RANGE']}
  • {'fieldName': 'Event Code', 'operators': ['BLANK', 'NOT_BLANK', 'EQUAL', 'NOT_EQUAL', 'IN_RANGE', 'NOT_IN_RANGE', 'CONTAIN', 'GREATER_THAN', 'LESS_THAN', 'GREATER_AND_EQUAL', 'LESS_AND_EQUAL']}
  • {'fieldName': 'Event Name', 'operators': ['BLANK', 'NOT_BLANK', 'EQUAL', 'NOT_EQUAL', 'IN_RANGE', 'NOT_IN_RANGE', 'CONTAIN', 'GREATER_THAN', 'LESS_THAN', 'GREATER_AND_EQUAL', 'LESS_AND_EQUAL']}
  • {'fieldName': 'Event Category', 'operators': ['BLANK', 'NOT_BLANK', 'EQUAL', 'NOT_EQUAL', 'IN_RANGE', 'NOT_IN_RANGE']}
  • {'fieldName': 'Event Topic', 'operators': ['BLANK', 'NOT_BLANK', 'EQUAL', 'NOT_EQUAL', 'IN_RANGE', 'NOT_IN_RANGE']}
  • {'fieldName': 'Event Archived', 'operators': ['EQUAL', 'NOT_EQUAL']}
  • {'fieldName': 'Event Campaign', 'operators': ['BLANK', 'NOT_BLANK', 'EQUAL', 'NOT_EQUAL

✓ Found 2 search fields

Sample search fields:
  • {'fieldName': 'Event', 'operators': ['BLANK', 'NOT_BLANK', 'EQUAL', 'NOT_EQUAL', 'IN_RANGE', 'NOT_IN_RANGE']}
  • {'fieldName': 'Event Code', 'operators': ['BLANK', 'NOT_BLANK', 'EQUAL', 'NOT_EQUAL', 'IN_RANGE', 'NOT_IN_RANGE', 'CONTAIN', 'GREATER_THAN', 'LESS_THAN', 'GREATER_AND_EQUAL', 'LESS_AND_EQUAL']}
  • {'fieldName': 'Event Name', 'operators': ['BLANK', 'NOT_BLANK', 'EQUAL', 'NOT_EQUAL', 'IN_RANGE', 'NOT_IN_RANGE', 'CONTAIN', 'GREATER_THAN', 'LESS_THAN', 'GREATER_AND_EQUAL', 'LESS_AND_EQUAL']}
  • {'fieldName': 'Event Category', 'operators': ['BLANK', 'NOT_BLANK', 'EQUAL', 'NOT_EQUAL', 'IN_RANGE', 'NOT_IN_RANGE']}
  • {'fieldName': 'Event Topic', 'operators': ['BLANK', 'NOT_BLANK', 'EQUAL', 'NOT_EQUAL', 'IN_RANGE', 'NOT_IN_RANGE']}
  • {'fieldName': 'Event Archived', 'operators': ['EQUAL', 'NOT_EQUAL']}
  • {'fieldName': 'Event Campaign', 'operators': ['BLANK', 'NOT_BLANK', 'EQUAL', 'NOT_EQUAL', 'IN_RANGE', 'NOT_IN_RANGE']}
  • 

In [6]:
# Get event output fields
print("📊 Retrieving event output fields...")

try:
    output_fields = client.events.get_output_fields()
    
    print(f"✓ Found {len(output_fields)} output fields")
    print("\nSample output fields:")
    for field in output_fields['standardFields']:
        print(f"  • {field}")
        
    for field in output_fields['customFields']:
        print(f"  • {field}")
        
except Exception as e:
    print(f"❌ Failed to retrieve output fields: {e}")

📊 Retrieving event output fields...
✓ Found 2 output fields

Sample output fields:
  • Address Created By
  • Address Created Date/Time
  • Address Last Modified By
  • Address Last Modified Date/Time
  • Address Line 1
  • Address Line 2
  • Address Line 3
  • Address Line 4
  • Address Type
  • Advantage Amount
  • Advantage Description
  • Attendee cap per registration
  • Campaign Code
  • Campaign End Date
  • Campaign Goal
  • Campaign ID
  • Campaign Name
  • Campaign Start Date
  • City
  • Company Type
  • Country
  • County
  • Declined
  • Deferred
  • Eligible Amount
  • Email Broadcast Date
  • Event Admission Fee
  • Event Archive
  • Event Capacity
  • Event Category Name
  • Event Code
  • Event Created By
  • Event Created Date/Time
  • Event Description
  • Event End Date
  • Event End Time
  • Event External URL
  • Event ID
  • Event Last Modified By
  • Event Last Modified Date/Time
  • Event Location Name
  • Event Name
  • Event Note
  • Event Registration Attend

## 5. Event Registration Analysis (Read-Only)

Analyze event registrations if events are available.

In [7]:
# Analyze event registrations
print("📈 Analyzing event registrations...")

if events:
    # Get the first event for demonstration
    sample_event = events[0]
    event_id = sample_event.get('id')
    
    if event_id:
        try:
            # Get registrations for this event
            registrations = list(client.events.get_registrations(event_id, limit=50))
            
            print(f"✓ Event '{sample_event.get('name')}' has {len(registrations)} registrations")
            
            if registrations:
                # Analyze registration data
                reg_df = pd.DataFrame(registrations)
                
                print("\n📊 Registration Summary:")
                print(f"Total Registrations: {len(registrations)}")
                
                # Show registration status distribution if available
                if 'registrationStatus' in reg_df.columns:
                    status_counts = reg_df['registrationStatus'].value_counts()
                    print("\nRegistration Status Distribution:")
                    for status, count in status_counts.items():
                        print(f"  {status}: {count}")
                        
            else:
                print("No registrations found for this event.")
                
        except Exception as e:
            print(f"❌ Failed to retrieve registrations: {e}")
    else:
        print("❌ Event ID not available.")
else:
    print("⚠️ No events available for registration analysis.")

📈 Analyzing event registrations...
✓ Event 'ECC Democrats Annual Holiday Party' has 1 registrations

📊 Registration Summary:
Total Registrations: 1


✓ Event 'ECC Democrats Annual Holiday Party' has 1 registrations

📊 Registration Summary:
Total Registrations: 1


## 6. Event Performance Metrics (Read-Only)

Calculate performance metrics for events.

In [8]:
# Calculate event performance metrics
print("📊 Calculating event performance metrics...")

if events:
    # Create a summary of all events
    event_summary = []
    
    for event in events[:5]:  # Analyze first 5 events for performance
        event_id = event.get('id')
        event_name = event.get('name', 'Unnamed Event')
        
        try:
            # Get registration count
            registrations = list(client.events.get_registrations(event_id, limit=1000))
            registration_count = len(registrations)
            
            # Calculate basic metrics
            event_summary.append({
                'Event Name': event_name,
                'Event ID': event_id,
                'Start Date': event.get('startDate', 'N/A'),
                'Status': event.get('eventStatus', 'N/A'),
                'Registration Count': registration_count,
                'Category': event.get('category', {}).get('name', 'N/A')
            })
            
        except Exception as e:
            print(f"⚠️ Could not analyze event {event_name}: {e}")
            continue
    
    if event_summary:
        # Display performance summary
        summary_df = pd.DataFrame(event_summary)
        
        print("\n📈 Event Performance Summary:")
        print(summary_df)
        
        print("\n📊 Key Metrics:")
        total_registrations = summary_df['Registration Count'].sum()
        avg_registrations = summary_df['Registration Count'].mean()
        
        print(f"Total Registrations Across Events: {total_registrations}")
        print(f"Average Registrations Per Event: {avg_registrations:.1f}")
        
        # Find most popular event
        if total_registrations > 0:
            top_event = summary_df.loc[summary_df['Registration Count'].idxmax()]
            print(f"Most Popular Event: {top_event['Event Name']} ({top_event['Registration Count']} registrations)")
    else:
        print("⚠️ Could not analyze any events.")
else:
    print("⚠️ No events available for performance analysis.")

📊 Calculating event performance metrics...

📈 Event Performance Summary:
                           Event Name  Event ID  Start Date Status  \
0  ECC Democrats Annual Holiday Party         1  2019-12-19    N/A   
1              SPRING CELEBRATION USE       761  2023-04-14    N/A   
2                  Summer Picnic 2023       763  2023-08-17    N/A   
3           ECCD Annual Holiday Party      1266  2023-12-14    N/A   
4             2024 Spring Celebration      1268  2024-04-15    N/A   

   Registration Count Category  
0                   1      N/A  
1                  88      N/A  
2                  38      N/A  
3                  42      N/A  
4                  90      N/A  

📊 Key Metrics:
Total Registrations Across Events: 259
Average Registrations Per Event: 51.8
Most Popular Event: 2024 Spring Celebration (90 registrations)



📈 Event Performance Summary:
                           Event Name  Event ID  Start Date Status  \
0  ECC Democrats Annual Holiday Party         1  2019-12-19    N/A   
1              SPRING CELEBRATION USE       761  2023-04-14    N/A   
2                  Summer Picnic 2023       763  2023-08-17    N/A   
3           ECCD Annual Holiday Party      1266  2023-12-14    N/A   
4             2024 Spring Celebration      1268  2024-04-15    N/A   

   Registration Count Category  
0                   1      N/A  
1                  88      N/A  
2                  38      N/A  
3                  42      N/A  
4                  90      N/A  

📊 Key Metrics:
Total Registrations Across Events: 259
Average Registrations Per Event: 51.8
Most Popular Event: 2024 Spring Celebration (90 registrations)


## ⚠️ Database-Modifying Operations (COMMENTED OUT FOR SAFETY)

The following operations would modify your database and are commented out for safety.
Uncomment and modify carefully if you need to create/update events.

In [9]:
# ⚠️ WARNING: DATABASE-MODIFYING OPERATIONS COMMENTED OUT FOR SAFETY

# # Create a new event (DANGEROUS - COMMENTED OUT)
# print("🆕 Creating new event...")
# 
# event_data = {
#     "event": {
#         "name": "Test Event - SDK Demo",
#         "startDate": (datetime.now() + timedelta(days=30)).strftime("%Y-%m-%d"),
#         "endDate": (datetime.now() + timedelta(days=30)).strftime("%Y-%m-%d"),
#         "eventStatus": "ACTIVE",
#         "category": {"id": 1},  # Use appropriate category ID
#         "description": "Demo event created by Neon CRM SDK"
#     }
# }
# 
# try:
#     # new_event = client.events.create(event_data)
#     # print(f"✓ Created event: {new_event.get('id')}")
#     pass
# except Exception as e:
#     print(f"❌ Event creation failed: {e}")

print("🛡️ Database-modifying operations are commented out for safety.")
print("Uncomment the code above if you need to create events.")

🛡️ Database-modifying operations are commented out for safety.
Uncomment the code above if you need to create events.


## Summary

This notebook demonstrates comprehensive event management operations:

### ✅ Completed Operations:
- Event listing and basic information retrieval
- Event search with filtering capabilities
- Event category exploration
- Field information discovery
- Registration analysis and metrics
- Performance summary calculations

### 🛡️ Safety Features:
- All read-only operations are safe to execute
- Database-modifying operations are commented out
- Error handling for missing data scenarios
- Validation bypass for testing field names

### 📝 Key Learnings:
- Events support both list() and search() operations
- Field names should use Display Name format (e.g., "Event Name")
- Some event fields may be optional and not populated for all events
- Event categories provide useful organizational structure

### 🔧 Best Practices Demonstrated:
- **Graceful Degradation**: Handling cases where events are not available
- **Data Validation**: Checking for required fields and handling missing data
- **Performance Analysis**: Using metrics to evaluate event success
- **User Experience**: Providing clear feedback about system capabilities

### 🚀 Advanced Applications:
- **Capacity Planning**: Using utilization data to optimize event sizing
- **Revenue Optimization**: Identifying high-performing event characteristics
- **Marketing Insights**: Understanding which events resonate with audiences
- **Resource Allocation**: Data-driven decisions about event investments

Events are a powerful tool for engagement and fundraising - this notebook provides the foundation for building sophisticated event management and analysis systems.